## **II - Data Preprocessing**
In some datasets, there are missing values, outliers, and categorical variables. These issues can affect the performance of the model. Therefore, it is important to preprocess the data before training the model. In this notebook, we will preprocess the data by filling the missing values, encoding the categorical variables, and scaling the features.


### **II.1 Importing the Libraries and Loading the Data**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

awards_players = pd.read_csv('../data/basketballPlayoffs/awards_players.csv')
coaches = pd.read_csv('../data/basketballPlayoffs/coaches.csv')
players = pd.read_csv('../data/basketballPlayoffs/players.csv')
players_teams = pd.read_csv('../data/basketballPlayoffs/players_teams.csv')
series_post = pd.read_csv('../data/basketballPlayoffs/series_post.csv')
teams = pd.read_csv('../data/basketballPlayoffs/teams.csv')
teams_post = pd.read_csv('../data/basketballPlayoffs/teams_post.csv')

### **II.2 Maping the Categorical Variables**
Map categorical values (teams, arenas, awards) to numerical codes for model training.

In [3]:
map_teams = {
    'ATL': 0, 'CHA': 1, 'CHI': 2, 'CLE': 3, 'CON': 4, 
    'DET': 5, 'HOU': 6, 'IND': 7, 'LAS': 8, 'MIA': 9, 
    'MIN': 10, 'NYL': 11, 'ORL': 12, 'PHO': 13, 'POR': 14, 
    'SAC': 15, 'SAS': 16, 'SEA': 17, 'UTA': 18, 'WAS': 19
}

map_teams_name = {
    'Atlanta Dream': 0, 'Charlotte Sting': 1, 'Chicago Sky': 2, 
    'Cleveland Rockers': 3, 'Connecticut Sun': 4, 'Detroit Shock': 5, 
    'Houston Comets': 6, 'Indiana Fever': 7, 'Los Angeles Sparks': 8, 
    'Miami Sol': 9, 'Minnesota Lynx': 10, 'New York Liberty': 11, 
    'Orlando Miracle': 12, 'Phoenix Mercury': 13, 'Portland Fire': 14, 
    'Sacramento Monarchs': 15, 'San Antonio Silver Stars': 16, 
    'Seattle Storm': 17, 'Utah Starzz': 18, 'Washington Mystics': 19
}

map_arenas = {
    'Philips Arena': 0, 'Charlotte Coliseum': 1, 'Time Warner Cable Arena': 2, 
    'UIC Pavilion': 3, 'Quicken Loans Arena': 4, 'Mohegan Sun Arena': 5, 
    'The Palace of Auburn Hills': 6, 'Compaq Center': 7, 'Toyota Center': 8, 
    'Conseco Fieldhouse': 9, 'Staples Center': 10, 'AmericanAirlines Arena': 11, 
    'Target Center': 12, 'Madison Square Garden (IV)': 13, 'Amway Arena': 14, 
    'US Airways Center': 15, 'Rose Garden Arena': 16, 'ARCO Arena (II)': 17, 
    'AT&T Center': 18, 'KeyArena at Seattle Center': 19, 
    'EnergySolutions Arena': 20, 'Verizon Center': 21
}

map_awards = {
    'All-Star Game Most Valuable Player': 0,
    'Coach of the Year': 1,
    'Defensive Player of the Year': 2,
    'Kim Perrot Sportsmanship Award': 3,
    'Kim Perrot Sportsmanship': 3,
    'Most Improved Player': 4,
    'Most Valuable Player': 5,
    'Rookie of the Year': 6,
    'Sixth Woman of the Year': 7,
    'WNBA Finals Most Valuable Player': 8,
    'WNBA All-Decade Team': 9,
    'WNBA All Decade Team Honorable Mention': 10
}

### **II.3 Plot Functions**
Define plot functions to visualize the data.

In [11]:
def plot_missing_values(df, title):
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0]  
    
    if len(missing_values) > 0:
        plt.figure(figsize=(10, 6))
        missing_values.plot(kind='bar')
        plt.title(f'Missing Values in {title}')
        plt.ylabel('Number of Missing Values')
        plt.xlabel('Columns')
        plt.xticks(rotation=45)
        plt.show()
    else:
        print(f'No missing values in {title}')

### **II.4 Players Data Preprocessing**

The Players dataset needs further preprocessing, including dealing with missing values, correcting invalid data, and converting player positions into numerical values.

In [ ]:
players['pos'] = players['pos'].replace({'G-C': 'C-G', 'F-G': 'G-F'})

positions = players['pos'].unique()
map_positions = {pos: i for i, pos in enumerate(positions)}

players['pos'] = players['pos'].map(map_positions)

Remove players with invalid death dates and unnecessary columns, filter players without birth dates, identify relevant players in the teams table, and drop those with missing birth dates.

In [ ]:
players['birthDate'] = pd.to_datetime(players['birthDate'], errors='coerce')

players_cleaned = players[players['deathDate'] == '0000-00-00'].drop(columns=['deathDate','firstseason','lastseason'])

players_cleaned = players_cleaned.drop(columns=['collegeOther'])

players_no_birthdate = players_cleaned[players_cleaned['birthDate'].isnull()]

players_no_birthdate_teams = players_no_birthdate[players_no_birthdate['bioID'].isin(players_teams['playerID'])]

print("\nPlayers with no birthdate that are in the teams_players table:")

print(players_no_birthdate_teams)

indices_to_rem = players_no_birthdate.index

players_cleaned = players_cleaned.drop(indices_to_rem)

print(players_cleaned)

Check for missing values in the datasets.

In [ ]:
plot_missing_values(players_cleaned, 'Players')

### **II.5 Coaches Data Preprocessing**
In the Coaches dataset, we apply a similar mapping to team IDs.

In [9]:
coaches['tmID'] = coaches['tmID'].map(map_teams)

Drop the columns that are not needed for the analysis.

In [10]:
coaches_cleaned = coaches.drop(columns=['lgID'])

Check for missing values in the datasets.

In [ ]:
print("\nMissing Values in Coaches:")
print(coaches_cleaned.isnull().sum())

### **II.6 Teams Data Preprocessing**

In the Teams dataset, we will map some values (like team names and playoff outcomes) to numerical representations.

In [ ]:
#pd.set_option('future.no_silent_downcasting', True)

# Função para substituir valores
def replace_values(df, column, old_values, new_values):
    return df[column].replace(dict(zip(old_values, new_values)))


teams['firstRound'] = replace_values(teams, 'firstRound', ['L', 'W'], [0, 1])
teams['semis'] = replace_values(teams, 'semis', ['L', 'W'], [0, 1])
teams['finals'] = replace_values(teams, 'finals', ['L', 'W'], [0, 1])
teams['playoff'] = replace_values(teams, 'playoff', ['N', 'Y'], [0, 1])
teams['confID'] = replace_values(teams, 'confID', ['EA', 'WE'], [0, 1])
teams['tmID'] = teams['tmID'].replace(map_teams)
teams['franchID'] = teams['franchID'].replace(map_teams)
teams['name'] = teams['name'].replace(map_teams_name)
teams['arena'] = teams['arena'].replace(map_arenas)

Drop the columns that are not needed for the analysis.

In [9]:
teams_cleaned = teams.drop(columns=['lgID'])

Check for missing values in the datasets.

In [ ]:
print("\nMissing Values in Teams:")
print(teams_cleaned.isnull().sum())

plot_missing_values(teams_cleaned, 'Teams')

Drop the columns with missing values

In [17]:
teams_cleaned.drop(columns=['franchID','divID'])

#Assuming that the missing values in firstRound, semis and finals represent that the team didn't reach that stage we will replace all the missing values by the number 2
teams_cleaned['firstRound'] = teams_cleaned['firstRound'].fillna(2)
teams_cleaned['semis'] = teams_cleaned['semis'].fillna(2)
teams_cleaned['finals'] = teams_cleaned['finals'].fillna(2)

### **II.7 Players Teams Data Preprocessing**

In the Players Teams dataset, we will map some values (like team names) to numerical representations.

In [18]:
players_teams['tmID'] = players_teams['tmID'].map(map_teams)

Drop the columns that are not needed for the analysis.

In [19]:
players_teams_cleaned = players_teams.drop(columns=['lgID'])

Check for missing values in the datasets.

In [ ]:
print("\nMissing Values in Players Teams:")
print(players_teams_cleaned.isnull().sum())

### **II.8 Player Awards Data Preprocessing**

In the Awards Players dataset, we will map some values (like team names) to numerical representations.

In [22]:
awards_players['award'].replace(map_awards, inplace=True)

Drop the columns that are not needed for the analysis.

In [23]:
awards_players_cleaned = awards_players.drop(columns=['lgID'])

Check for missing values in the dataset.

In [ ]:
print("Missing Values in Awards Players:")
print(awards_players_cleaned.isnull().sum())

### **II.9 Series Post Data Preprocessing**
In the Series Post dataset, we will map some values (like playoff outcomes) to numerical representations.

In [25]:
series_post['tmIDWinner'] = series_post['tmIDWinner'].map(map_teams)
series_post['tmIDLoser'] = series_post['tmIDLoser'].map(map_teams)

Drop the columns that are not needed for the analysis.

In [26]:
series_post_cleaned = series_post.drop(columns=['lgIDWinner','lgIDLoser'])

Check for missing values in the datasets.

In [ ]:
print("\nMissing Values in Series Post:")
print(series_post_cleaned.isnull().sum())

### **II.10 Teams Post Data Preprocessing**
In the **Teams Post** dataset, we will map some values (like playoff outcomes) to numerical representations.

In [28]:
teams_post['tmID'] = teams_post['tmID'].map(map_teams)

Drop the columns that are not needed for the analysis.

In [29]:
teams_post_cleaned = teams_post.drop(columns=['lgID'])

Check for missing values in the datasets.

In [ ]:
print("\nMissing Values in Teams Post:")
print(teams_post_cleaned.isnull().sum())

### **II.11 Creating Output Directory and Exporting Cleaned DataFrames to new CSV files**
After preprocessing, export each cleaned dataset to new CSV files. This saves the refined data, making it easier to load for future analysis and model training.

In [ ]:
os.makedirs('../data/basketballPlayoffs_cleaned', exist_ok=True)

awards_players_cleaned.to_csv('../data/basketballPlayoffs_cleaned/awards_players_cleaned.csv', index=False)
coaches_cleaned.to_csv('../data/basketballPlayoffs_cleaned/coaches_cleaned.csv', index=False)
players_teams_cleaned.to_csv('../data/basketballPlayoffs_cleaned/players_teams_cleaned.csv', index=False)
players_cleaned.to_csv('../data/basketballPlayoffs_cleaned/players_cleaned.csv', index=False)
series_post_cleaned.to_csv('../data/basketballPlayoffs_cleaned/series_post_cleaned.csv', index=False)
teams_post_cleaned.to_csv('../data/basketballPlayoffs_cleaned/teams_post_cleaned.csv', index=False)
teams_cleaned.to_csv('../data/basketballPlayoffs_cleaned/teams_cleaned.csv', index=False)

### **II.12 Dealing with players weight missing data**
Lets import the new cleaned dataset so we can deal with the missing data

In [33]:
players_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/players_cleaned.csv')

This code calculates the weight of players using an ideal Body Mass Index (BMI) of 22 and rounding the resulting weights to integers.

In [34]:
ideal_bmi = 22

players_cleaned['height_m'] = players_cleaned['height'] * 0.0254

players_cleaned.loc[players_cleaned['weight'] == 0, 'weight'] = (ideal_bmi * (players_cleaned['height_m'] ** 2) / 0.453592).round(0)

players_cleaned['weight'] = players_cleaned['weight'].astype(int)

players_cleaned.drop(columns=['height_m'], inplace=True)

Export updated dataframe with weights

In [35]:
players_cleaned.to_csv('../data/basketballPlayoffs_cleaned/players_cleaned.csv', index=False)